In [1]:
include("../src/variationaltimedisc.jl")
using GradientRobustMultiPhysics
using ExtendableGrids
using Printf
using ExtendableSparse
using DelimitedFiles
using SimplexGridFactory
using Triangulate

In [38]:
function get_flow_data(ν)
    ## note that dependencies "XT" marks the function to be x- and t-dependent
    ## that causes the solver to automatically reassemble associated operators in each time step
    u = DataFunction((result, x, t) -> (
        result[1] = t*x[1]; 
        result[2] = -t*x[2];
        ), [2,2]; name = "u", dependencies = "XT", bonus_quadorder = 5)
    u_t = DataFunction((result, x, t) -> (
        result[1] = x[1];
        result[2] = -x[2];
        ), [2,2]; name = "u", dependencies = "XT", bonus_quadorder = 5)
    p = DataFunction((result, x) -> (
        result[1] = 0.0
        ), [1,2]; name = "p", dependencies = "X", bonus_quadorder = 5)
  
    ############## common code for all examples #####
    dt_u = eval_dt(u)
    Δu = eval_Δ(u)
    ∇p = eval_∇(p)
    ∇u = eval_∇(u)
    f = DataFunction((result, x, t) -> (
          result .= dt_u(x,t) .+ ∇u(x,t)*u(x,t) .- ν*Δu(x,t) .+ view(∇p(x,t),:);
          #result[1] += x[1];
          #result[2] += x[2];
        ), [2,2]; name = "f", dependencies = "XT", bonus_quadorder = 5)  
    return u, p, f, u_t
  end

get_flow_data (generic function with 1 method)

In [52]:
function run_main(; time_order=1, scheme = 0, ν=1, newton = false)
    nlevels = 2
    xgrid = simplexgrid(Triangulate;
            points=[0 0 ; 0 1 ; 1 1 ; 1 0]',
            bfaces=[1 2 ; 2 3 ; 3 4 ; 4 1 ]',
            bfaceregions=[1, 2, 3, 4],
            regionpoints=[0.5 0.5;]',
            regionnumbers=[1],
            regionvolumes=[4.0^(-nlevels-1)/2])
  @show xgrid
  FETypes = [H1P2{2,2}, H1P1{1}]

  u, p, f, dt_u = get_flow_data(ν)

  TDisc = VariationalTimeDisc.SetVariationalTimeDisc(Float64,time_order, scheme)  
  FES = [FESpace{FETypes[1]}(xgrid), FESpace{FETypes[2]}(xgrid)]
  # solution vector
  Solution = FEVector(["u_h", "p_h"], FES)
  @show Solution
  ndofu = FES[1].ndofs
  ndofp = FES[2].ndofs
  GradientRobustMultiPhysics.interpolate!(Solution[1], u; time = 0.)
  initSol = zeros(Float64, ndofu)
  initSol = Solution.entries # full solution

  # mass matrix 
  M = FEMatrix{Float64}(FES)
  assemble_operator!(M[1,1], BilinearForm([Identity, Identity]))
  # velocity pressure matrices
  A = FEMatrix{Float64}(FES)
  assemble_operator!(A[1,1], LaplaceOperator(ν))
  assemble_operator!(A[1,2], LagrangeMultiplier(Divergence); At = A[2,1]) 

  rhs = FEVector{Float64}(FES)
  assemble_operator!(rhs[1], LinearForm(Identity, f); time=0.0)
  """
  parameters used for the time steeping scheme
  """
  TPO = time_order # polynomial degree for time 
  Tk  = scheme  # scheme = 0 dG; scheme =1 cGP
  kL  = TDisc.kL
  kR  = TDisc.kR
  qp  = TDisc.p
  MassCoeffs = TDisc.MassCoeffs
  StiffCoeffs = TDisc.StiffCoeffs
  nQF = TDisc.nQF
  wQF = TDisc.wQF
  IC = TDisc.IC
  t0 = 0
  tau = 1/10

  V1 = zeros(Float64, ndofu, TPO+1)  
  SysFES = Array{FESpace{Float64, Int32}, 1}([])
  for j=1:TPO+1-Tk
    append!(SysFES, FES)
  end
  SystemMatrix = FEMatrix{Float64}(SysFES)
  SysNLMatrix = FEMatrix{Float64}(SysFES)
  SysNLRhs = FEVector{Float64}(SysFES)
  SystemRHS = FEVector{Float64}(SysFES)
  SystemSol = FEVector{Float64}(SysFES)
  # number of solution vectors to be calculated
  d = TPO+1-kL
  # Number of inner solution vectors
  di = d-kR
  @show TPO, Tk, kL, kR, d, di 
  dt = Array{BoundaryData,1}(undef,0)
  push!(dt,BoundaryData(BestapproxDirichletBoundary; regions = [1,2,3,4], data=u))
  
  ## the convection operator is assembled to the right-hand side
  ## to keep the matrix constant in time (but we do subiterations in each timestep)
  if (newton)
    function convection_kernel(result, input)
        uh, ∇uh = view(input,1:2), view(input,3:6)
        result[1] = ∇uh[1]*uh[1] + ∇uh[2]*uh[2]
        result[2] = ∇uh[3]*uh[1] + ∇uh[4]*uh[2]
    end
    CO = NonlinearForm(Identity, [Identity, Gradient], [1,1], convection_kernel, [2,6]; 
          name = "((#1⋅∇)#1, #T)")
    DCO = GradientRobustMultiPhysics.AssemblyPattern{GradientRobustMultiPhysics.APT_NonlinearForm, 
          Float64, ON_CELLS}(CO.name, [FES[1], FES[1], FES[1]], CO.operators4arguments,
          CO.action,CO.apply_action_to,CO.regions)
    DCO.newton_args = CO.newton_arguments  
    @show DCO.newton_args
  else
    # disctrete convection operator
    # Picard iteration
    CO = ConvectionOperator(1, Identity, 2, 2; newton = newton)
    DCO = GradientRobustMultiPhysics.AssemblyPattern{GradientRobustMultiPhysics.APT_BilinearForm, 
          Float64, ON_CELLS}(CO.name, [FES[1], FES[1], FES[1]], CO.operators4arguments,CO.action,
          CO.apply_action_to,CO.regions)
  end

  # assemble the nonlinear matrix and rhs
  ANL = FEMatrix{Float64}(FES)  
  rhsNL = FEVector{Float64}(FES)
  residual = FEVector{Float64}(SysFES)
  Tf = 1
  dofs = nothing
  while t0 <= Tf-1e-10
    # t0 = t0 + tau
    if kL > 0 # cGP 
      @show "No dG"
      V1[:, 1] = rhs.entries
    end
    # inner quad points
    for i = 1 : TPO - Tk
      fill!(rhs.entries, 0)
      assemble_operator!(rhs[1], LinearForm(Identity, f), time = t0 + tau * (qp[i]+1)/2)
      V1[:,i + kL] = rhs[1][:]
    end
    fill!(rhs.entries, 0)
    assemble_operator!(rhs[1], LinearForm(Identity, f), time = t0 + tau)
    # F_n rhs vector 
    V1[:, 1+(TPO+1-kR)] = rhs[1][:]
    Mu0 = zeros(Float64, ndofu)

    # mass matrix time the initial solution 
    addblock_matmul!(Mu0, M[1,1], Solution[1]) #; factor = 1) 
    # preparation of the system right hand side 
    fill!(SystemRHS.entries, 0)
    for i = 1 : TPO+1-Tk
      if Tk>0
        addblock!(SystemRHS[2*i-1], Mu0 * MassCoeffs[i, 1:kL]; factor= - 1.0)
        addblock!(SystemRHS[2*i-1], A.entries * Solution[1].entries; 
                  factor= - tau/2 * StiffCoeffs[i, 1])
      else
        # M * ξ 
        addblock!(SystemRHS[2*i-1], Mu0; factor= IC[i])
      end
    end
    for i= 1 : TPO+1-Tk
      addblock!(SystemRHS[2*i-1], V1 * StiffCoeffs[i, :]; factor= tau/2 )
    end    
    # reset the system matrix
    fill!(SystemMatrix.entries.cscmatrix.nzval, 0)
    # putting everything together for the left hand side 
    for s1 = 1 : di + 1
      fill!(A[1,1], 0)
      assemble_operator!(A[1, 1], LaplaceOperator(ν); time= t0 + tau * (qp[s1] +1) / 2. )
      for s2 = 1 : TPO + 1 - Tk
        addblock!(SystemMatrix[2*s2-1, 2*s1-1], M[1, 1]; factor= MassCoeffs[s2, s1+kL])
        if (s1 == s2)
          addblock!(SystemMatrix[2*s2-1, 2*s1-1], A[1, 1]; factor= StiffCoeffs[s2, s1+kL] * tau / 2)
          ## set the correct rows and cols
          addblock!(SystemMatrix[2*s2-1, 2*s1], A[1, 2]; factor= StiffCoeffs[s2, s1+kL] * tau / 2)
          addblock!(SystemMatrix[2*s2, 2*s1-1], A[2, 1]; factor= StiffCoeffs[s2, s1+kL] * tau / 2)
        end
      end
    end

    niter = 0
    res = 1e60
    flush!(SystemMatrix.entries)
    # Nonlinear loop 
    while niter <= 10 && res >= 1e-10
      fill!(SysNLMatrix.entries.cscmatrix.nzval, 0)
      SysNLMatrix.entries.cscmatrix += SystemMatrix.entries.cscmatrix
      fill!(SysNLRhs.entries, 0)
      SysNLRhs.entries .+= SystemRHS.entries
      flush!(SysNLMatrix.entries)
      for s1 = 1 : di +1 
        if newton
          GradientRobustMultiPhysics.full_assemble!(SysNLMatrix[2*s1-1, 2*s1-1], SysNLRhs[2*s1-1], 
            DCO, [SystemSol[2*s1-1], SystemSol[2*s1-1]]; skip_preps = false, factor = CO.factor * StiffCoeffs[s1, s1+kL] * tau / 2, 
            transposed_assembly = CO.transposed_assembly)        
        else
          # Assemble only ANL for Picard iteration 
          GradientRobustMultiPhysics.assemble!(SysNLMatrix[2*s1-1, 2*s1-1], DCO, [SystemSol[2*s1-1], SystemSol[2*s1-1]]; 
            factor = StiffCoeffs[s1, s1+kL] * tau / 2, transposed_assembly=true)
        end
      end      
      for i = 1 : TPO + 1 - Tk
        dofs = boundarydata!(SystemSol[2*i-1], dt; time = t0 + tau * (qp[i]+1)/2)
        # println("dofs: ", dofs)
        for dof in dofs
          SysNLRhs[2*i-1][dof] = 1e60 * SystemSol[2*i-1][dof]
          SysNLMatrix[2*i-1,2*i-1][dof,dof] = 1e60
        end
        SysNLMatrix[2*i,2*i][1,1] = 1e60 #pressure correction        
      end
      flush!(SysNLMatrix.entries)
      GradientRobustMultiPhysics.mul!(residual.entries,SysNLMatrix.entries,SystemSol.entries)
      residual.entries .-= SysNLRhs.entries
      for i = 1 : TPO + 1- Tk  
        for dof in dofs 
          residual[2*i - 1][dof] = 0  
        end        
        residual[2*i][1] = 0
      end      
      res = norm(residual.entries)
      @info (niter, res)
      SystemSol.entries[:] = SysNLMatrix.entries \ SysNLRhs.entries 
      
      niter = niter + 1
    end#end while nonlinear loop
    for j = 1 : ndofu 
      Solution[1][j] = SystemSol[2*di+1][j]
    end
    L2Error_u = L2ErrorIntegrator(u, Identity; time= t0 + tau)
    l2 = evaluate(L2Error_u, Solution[1])
    @info sqrt(l2)
    t0 = t0 + tau
  end # endwhile time loop
end

run_main (generic function with 1 method)

In [53]:
run_main(time_order=1)

xgrid = ExtendableGrid{Float64, Int32};
dim: 2 nodes: 115 cells: 194 bfaces: 34



VariationalTimeDisc(r=1, k=0): dG(1)
Solution = 
FEVector information
   block  |  ndofs  | name (FEType) 
 [    1]  |     846 | u_h (H1P2{2,2})
 [    2]  |     115 | p_h (H1P1{1})



(TPO, Tk, kL, kR, d, di) = (1, 0, 0, 1, 2, 1)


┌ Info: (0, 0.06890817566810066)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: (1, 2.3680115649939488e-5)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192


┌ Info: (2, 1.3361870439423658e-11)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: 6.280984627477984e-16
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:202


┌ Info: (0, 0.09079469282942222)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: (1, 5.6571777309084745e-5)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192


...

┌ Info: (0, 0.09091896504814063)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: (1, 0.00028867389335809917)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192


┌ Info: (2, 1.444196753359082e-9)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: (3, 1.0730835074692314e-12)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192


┌ Info: 2.563014861924999e-15
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:202
┌ Info: (0, 0.09093796777393019)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192


┌ Info: (1, 0.0003219357635231446)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: (2, 1.7950961969350985e-9)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192


┌ Info: (3, 1.4855024035154906e-12)
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:192
┌ Info: 3.8846644930856865e-15
└ @ Main /home/ahmed/OneDrive/ResearchWork/codes/JuliaFE_Julia23/my_examples/TimeNavierStokesVTD.ipynb:202
